In [1]:
#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))  
import IPython.display as display

/var/folders/ry/454yhlzx6hd15j7rjv4th0lw0000gn/T/ipykernel_90169/4123656545.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [8]:
from joblib import dump, load
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [12]:
# Load your training data
x_test, y_test = load('/Users/ciprian/Desktop/Projects/Smart Plant Pot/Software/ai_smart_plant_pot/Python/saved_data/data/adversarial-training_medium-masked_data/test_data.joblib')
x_test = np.array(x_test, dtype=np.float32)

In [14]:
# Define GradientReversalLayer for adversarial models
class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self, lambda_, **kwargs):
        super(GradientReversalLayer, self).__init__(**kwargs)
        self.lambda_ = lambda_

    @tf.custom_gradient
    def call(self, x):
        def grad(dy):
            return -self.lambda_ * dy
        return x, grad

    def get_config(self):
        config = super().get_config()
        config.update({
            "lambda_": self.lambda_
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# Custom objects dictionary
custom_objects = {"GradientReversalLayer": GradientReversalLayer}

# Load the model
model = load_model("saved_data/models/adversarial-training_medium-masked_mobilenetv2_fine-tuning_v3/adversarial-training_medium-masked_mobilenetv2_fine-tuning.keras", custom_objects=custom_objects)

# Convert the model to TensorFlow Lite format with full integer quantization and TF Select
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Provide the representative dataset
def representative_dataset():
    for i in range(100):  # Adjust the number of samples as needed
        data = x_test[i:i+1]  # Yield one sample at a time
        yield [tf.convert_to_tensor(data, dtype=tf.float32)]

converter.representative_dataset = representative_dataset

# Specify the target specification to use only integer operations
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.SELECT_TF_OPS]
converter.inference_input_type = tf.uint8  # or tf.int8
converter.inference_output_type = tf.uint8  # or tf.int8

tflite_quant_model = converter.convert()

# Save the quantized model
with open('quantized_adversarial_model_int8.tflite', 'wb') as f:
    f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: /var/folders/ry/454yhlzx6hd15j7rjv4th0lw0000gn/T/tmpgoe7soew/assets


INFO:tensorflow:Assets written to: /var/folders/ry/454yhlzx6hd15j7rjv4th0lw0000gn/T/tmpgoe7soew/assets
/Users/ciprian/Documents/anaconda/anaconda3/envs/tf_2.1/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-05-22 05:03:51.827593: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-05-22 05:03:51.827610: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-05-22 05:03:51.827745: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/ry/454yhlzx6hd15j7rjv4th0lw0000gn/T/tmpgoe7soew
2024-05-22 05:03:51.840809: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-22 05:03:51.840825: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/ry

fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: UINT8
